# Lab 8, Module 4: Professional Diffusion Models

**Estimated time:** 12-15 minutes

---

## **Opening: From Toy to Professional**

In Modules 2-3, you trained a toy model (~100k parameters) that generates 16×16 grayscale digits.

The results were recognizable but blurry. Why?
- **Small model:** Only 100,000 parameters
- **Limited data:** 24,000 MNIST digits
- **Low resolution:** 16×16 pixels
- **Short training:** 2-3 minutes

Now you'll see what happens when we **scale up**!

In this module, you'll:
- Use a **pre-trained professional model** (~30 million parameters)
- Generate **32×32 RGB images** (full color!)
- Compare **inference steps** (25 vs 50 vs 100)
- Understand the **quality-speed trade-off**
- Learn how to bridge to **text-to-image models** (DALL-E, Stable Diffusion)

**Most importantly:** You'll see that professional models use the **EXACT SAME algorithm** you implemented in Modules 2-3!

The difference is just **scale**: more data, bigger model, longer training.

---

## 🔧 **A Note About PyTorch**

In Modules 1-3, we used **Keras/TensorFlow** to build our toy diffusion model.

In Module 4, we'll use **PyTorch** instead—another popular deep learning framework.

**Why the switch?**
- The Hugging Face `diffusers` library (industry standard) works best with PyTorch
- PyTorch and TensorFlow are just different tools for the same job (like Word vs. Google Docs)
- Both frameworks implement the same concepts: tensors, gradients, neural networks

**Don't worry!** You don't need to understand PyTorch syntax. We're using it like a black box—the same diffusion concepts from Modules 1-3 still apply!

**The key point:** The algorithm (DDPM reverse diffusion) is identical. Only the syntax changes slightly.

---

## 📦 **Setup: Install and Import Libraries**

First, let's install the `diffusers` library and import what we need.

In [ ]:
# Install the diffusers library (takes ~30 seconds)
!pip install -q diffusers transformers accelerate

print("✅ Installation complete!")

In [ ]:
# Imports
import torch
from diffusers import DDPMPipeline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

print(f"✅ PyTorch version: {torch.__version__}")
print("✅ Libraries imported!")

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🖥️  Using device: {device}")

if device == "cpu":
    print("\n⚠️  Warning: Running on CPU will be slower (~2-3 minutes per generation)")
    print("    Consider enabling GPU in Runtime > Change runtime type > T4 GPU")
else:
    print("\n🚀 GPU detected! Generation will be fast (~30-60 seconds per batch).")

---

## 📥 **Load Pre-Trained Model**

We'll load a professional diffusion model trained on **CIFAR-10**:
- **Dataset:** 60,000 images of 10 object categories
- **Categories:** airplane, car, bird, cat, deer, dog, frog, horse, ship, truck
- **Resolution:** 32×32 RGB (color images)
- **Parameters:** ~30 million (300× more than our toy model!)
- **Training time:** Days/weeks on multiple GPUs

This model was trained by Google and is available on Hugging Face.

In [ ]:
# Load the pre-trained CIFAR-10 diffusion model (~200 MB download)
print("📥 Downloading model (this may take 1-2 minutes)...\n")
pipeline = DDPMPipeline.from_pretrained("google/ddpm-cifar10-32")
pipeline = pipeline.to(device)

print("\n✅ Model loaded successfully!")
print(f"\n📊 Model details:")
print(f"   - Training dataset: CIFAR-10 (60,000 images)")
print(f"   - Image size: 32×32 pixels (RGB color)")
print(f"   - Object classes: 10 (airplane, car, bird, cat, deer, dog, frog, horse, ship, truck)")
print(f"   - Parameters: ~30 million (vs. ~100,000 in our toy model!)")
print(f"\n💡 This model uses the SAME DDPM algorithm you implemented in Module 3!")

---

## 🎨 **Generate Images from Noise**

Let's generate a batch of 16 images from pure random noise!

**What's happening behind the scenes:**
1. Start with 16 random noise tensors (32×32×3 each)
2. Run 50 denoising steps (same reverse diffusion you implemented!)
3. Each step: predict noise, remove it, add tiny random noise
4. Result: 16 generated images of random objects

This takes ~30-60 seconds with GPU, 2-3 minutes with CPU.

In [ ]:
# Generate a grid of 16 images
print("🎨 Generating 16 images from pure noise...")
print("   (This takes ~30-60 seconds with GPU, 2-3 minutes with CPU)\n")

# Set random seed for reproducibility
torch.manual_seed(42)

# Generate images using 50 denoising steps
output = pipeline(
    batch_size=16,
    num_inference_steps=50,
    output_type="pil"
)
images = output.images

print("\n✅ Generation complete!")
print(f"   Generated {len(images)} images")
print(f"   Each image: 32×32 pixels, RGB color")

---

## 📸 **Visualize Generated Images**

Let's see what the model created!

In [ ]:
# Display in 4×4 grid
fig, axes = plt.subplots(4, 4, figsize=(10, 10))
fig.suptitle("Professional Diffusion Model: CIFAR-10 Generation\n(50 denoising steps)",
             fontsize=14, fontweight='bold')

for idx, ax in enumerate(axes.flat):
    ax.imshow(images[idx])
    ax.axis('off')
    ax.set_title(f"Sample {idx+1}", fontsize=9)

plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("   - Images are 32×32 RGB (color)")
print("   - Objects are recognizable (cars, planes, animals)")
print("   - Quality is MUCH better than our 16×16 toy model")
print("   - This model trained on 60,000 images for days/weeks")
print("\n✓ Same algorithm, dramatically better results due to scale!")

---

## 📝 **Question Q21 (Observation)**

### **Q21. Look at the 16 generated images. How many distinct object categories can you identify?**

*(Hint: CIFAR-10 has 10 classes: airplane, car, bird, cat, deer, dog, frog, horse, ship, truck)*

**Record your answer in the Answer Sheet.**

---

## 🔬 **Experiment: Step Count Comparison**

In Module 3, you saw that more denoising steps generally improve quality.

Let's test this with the professional model!

**Question:** Does 100 steps produce noticeably better results than 25 steps?

**Trade-off:**
- **Fewer steps:** Faster generation, possibly lower quality
- **More steps:** Slower generation, possibly better quality

Let's find out!

In [ ]:
# Compare different numbers of denoising steps
step_counts = [25, 50, 100]
comparison_images = []

print("🔬 Experiment: How do step counts affect quality?\n")

for steps in step_counts:
    print(f"Generating with {steps} steps...", end='\r')
    torch.manual_seed(42)  # Same seed for fair comparison
    output = pipeline(
        batch_size=1,
        num_inference_steps=steps,
        output_type="pil"
    )
    comparison_images.append(output.images[0])
    print(f"   ✓ Done ({steps} steps)")

print("\n✅ Comparison complete!")

---

## 📊 **Visualize Step Comparison**

Let's see the quality differences side-by-side!

In [ ]:
# Side-by-side comparison
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
fig.suptitle("Quality vs. Speed Trade-off: Inference Steps",
             fontsize=14, fontweight='bold')

for idx, (steps, img) in enumerate(zip(step_counts, comparison_images)):
    axes[idx].imshow(img)
    axes[idx].set_title(f"{steps} steps", fontsize=12, fontweight='bold')
    axes[idx].axis('off')

    # Color-coded borders
    if steps == 25:
        color = 'orange'
        label = 'Faster, lower quality'
    elif steps == 50:
        color = 'green'
        label = 'Balanced'
    else:
        color = 'blue'
        label = 'Slower, best quality'

    for spine in axes[idx].spines.values():
        spine.set_edgecolor(color)
        spine.set_linewidth(3)

    axes[idx].text(0.5, -0.15, label, transform=axes[idx].transAxes,
                   ha='center', fontsize=9, style='italic')

plt.tight_layout()
plt.show()

print("\n💡 Key Insight:")
print("   - More steps = higher quality but slower generation")
print("   - 25 steps: Fast but slightly noisier (~10-15 seconds)")
print("   - 50 steps: Good balance (~30-60 seconds)")
print("   - 100 steps: Best quality but slowest (~1-2 minutes)")
print("   - DALL-E/Stable Diffusion typically use 20-50 steps")
print("\n✓ Diminishing returns: 50→100 steps improves less than 25→50 steps")

---

## 📝 **Questions Q22-Q23 (Analysis)**

### **Q22. Compare the images generated with 25, 50, and 100 steps. Describe the quality differences you observe. Is the improvement from 50→100 steps as noticeable as 25→50 steps?**

**Record your answer in the Answer Sheet.**

---

### **Q23. Why might DALL-E and Stable Diffusion use 20-50 steps instead of 100+ steps? What's the trade-off?**

*(Hint: Consider both user experience and computational cost)*

**Record your answer in the Answer Sheet.**

---

## 📊 **Comparison: Toy Model vs. Professional Model**

Let's compare what you built in Modules 2-3 to this pre-trained professional model:

| Aspect | Module 3 Toy Model | Module 4 Professional |
|--------|-------------------|----------------------|
| **Training Data** | 24,000 MNIST digits | 60,000 CIFAR-10 images |
| **Image Size** | 16×16 grayscale | 32×32 RGB color |
| **Channels** | 1 (grayscale) | 3 (red, green, blue) |
| **Classes** | 4 digits (0-3) | 10 object categories |
| **Parameters** | ~100,000 | ~30 million |
| **Training Time** | 2-3 minutes (Colab T4) | Days/weeks (multiple GPUs) |
| **Quality** | Blurry but recognizable | Sharp, detailed objects |
| **Algorithm** | DDPM reverse diffusion | **SAME ALGORITHM!** |

### **The Key Takeaway:**
- ✅ You implemented the **EXACT SAME algorithm** as professional models
- ✅ The difference is **scale**: more data, bigger model, longer training
- ✅ The core diffusion process (forward noise, reverse denoising) is **identical**!

**This is incredibly powerful:**
- You understand the foundation of DALL-E, Midjourney, and Stable Diffusion
- The algorithm doesn't change—only the scale increases
- Professional models just have billions of parameters instead of millions

---

## 📝 **Question Q24 (Synthesis)**

### **Q24. Looking at the comparison table, what specific factors contribute most to the quality difference between your toy model and the professional CIFAR-10 model?**

*(Consider: parameters, training data, resolution, training time—which matter most?)*

**Record your answer in the Answer Sheet.**

---

## 🌉 **From CIFAR-10 to DALL-E**

### **What We've Seen So Far**

**Module 3 & 4: Unconditional Generation**
- **Input:** Pure random noise
- **Output:** Random image from learned distribution
- **No control** over what gets generated

**Example:** CIFAR-10 model generates random objects (could be a car, a bird, a cat, etc.)

---

### **What DALL-E and Stable Diffusion Add**

**Text-to-Image: Conditional Generation**
- **Input:** Text prompt ("a cat wearing an astronaut suit on Mars")
- **Output:** Image matching the description
- **Full creative control**

---

### **How Text Conditioning Works (Simplified)**

```
1. Text Encoder: "a red sports car" → embedding vector (list of numbers)
2. Diffusion Model: Uses text embedding to guide denoising
   - At each step, the model asks: "Does this look like 'a red sports car'?"
   - Adjusts denoising to match the text description
3. Output: Image of a red sports car
```

**Three-Tier Explanation:**

**Plain English:**  
The model learns to ask "What object am I making?" at each denoising step, using the text description as a guide.

**Symbols:**  
x_{t-1} = Denoise(x_t, t, text_embedding)

**Pseudocode:**
```python
for t in reversed(range(T)):
    noise_prediction = model(noisy_image, timestep=t, text_embedding=text_embed)
    noisy_image = denoise_one_step(noisy_image, noise_prediction)
```

**The key addition:** The denoising model receives **two inputs** instead of one:
1. Noisy image (x_t)
2. **Text embedding** (what we want to generate)

The model learns: "Remove noise in a way that makes this look like [text description]"

---

### **Scaling Up: DALL-E vs. Our Models**

| Model | Parameters | Training Data | Capabilities |
|-------|-----------|---------------|--------------||
| **Our Toy Model (Module 3)** | 100K | 24K MNIST digits | Generates blurry 0-3 digits |
| **CIFAR-10 (Module 4)** | 30M | 60K small objects | Generates 32×32 objects |
| **Stable Diffusion** | 890M | 2B+ text-image pairs | Generates 512×512 images from text |
| **DALL-E 3** | ~10B+ | Billions of images | Photorealistic images from complex prompts |

**Same algorithm, different scale!**

---

## 🌍 **Real-World Applications**

Diffusion models power many cutting-edge AI systems:

### **Creative Applications**
- **DALL-E 3** (OpenAI): Text-to-image generation with unprecedented quality
- **Midjourney**: Artistic image generation favored by designers and artists
- **Stable Diffusion**: Open-source text-to-image (anyone can run it!)
- **Adobe Firefly**: Creative tools integrated into Photoshop and Illustrator

### **Beyond Images**
- **Video generation:** Runway, Pika Labs, Sora (text-to-video)
- **Medical imaging:** Enhancing MRI/CT scans, generating synthetic training data
- **Drug discovery:** Generating novel molecular structures for pharmaceuticals
- **Audio:** Music generation, voice synthesis, sound effects
- **3D models:** Generating 3D objects and scenes from text descriptions

### **Why Diffusion Models Matter**
- **High quality:** Better than previous generative methods (GANs, VAEs)
- **Versatile:** Work for images, video, audio, 3D, molecules, and more
- **Controllable:** Text conditioning provides precise creative control
- **Iterative:** Step-by-step refinement produces detailed, high-quality results
- **Stable training:** Easier to train than GANs, more reliable convergence

### **The Diffusion Revolution**
- **2020:** DDPM paper introduces the algorithm you learned
- **2022:** DALL-E 2 and Stable Diffusion launch publicly
- **2023:** Billions of images generated, mainstream adoption
- **2024:** Video generation (Sora), 3D generation, and beyond

**You now understand the algorithm powering this revolution!**

---

## 📝 **Question Q25 (Synthesis & Reflection)**

### **Q25. In your own words, explain the complete diffusion pipeline: How does a model like DALL-E go from a text prompt ('a cat in a spacesuit') to a final image? Reference concepts from Modules 0-4.**

*(Think about: text encoding, noise initialization, reverse diffusion, text conditioning)*

**Record your answer in the Answer Sheet.**

---

---

## ✅ **Module 4 Complete!**

### **What You Learned**

In this module, you:
- ✅ Used a professional pre-trained diffusion model (30M parameters)
- ✅ Generated high-quality 32×32 RGB images
- ✅ Compared inference steps (25 vs 50 vs 100)
- ✅ Saw the quality improvement from scale
- ✅ Learned how text-to-image conditioning works
- ✅ Connected to DALL-E, Stable Diffusion, and real-world applications

### **Key Insights**

> **The algorithm you implemented in Modules 2-3 is the SAME as DALL-E and Stable Diffusion!**
>
> The difference is scale:
> - More training data (billions vs. thousands)
> - Bigger models (billions vs. millions of parameters)
> - Longer training (weeks/months vs. minutes)
> - Text conditioning (guides generation toward descriptions)

---

## 🎉 **Lab 8 Complete!**

Congratulations! You've completed all 5 modules of Lab 8!

### **Your Journey:**

1. **Module 0:** Learned what diffusion models are (conceptual foundation)
2. **Module 1:** Saw how forward diffusion destroys information progressively
3. **Module 2:** Trained your own denoiser using a simplified U-Net
4. **Module 3:** Implemented reverse diffusion to generate images from noise
5. **Module 4:** Used a professional model and bridged to DALL-E

### **What You Now Understand:**

- ✅ **Forward diffusion:** Gradually adding noise destroys information
- ✅ **Denoising networks:** U-Net architectures predict noise at each timestep
- ✅ **Reverse diffusion:** Iterative denoising creates images from noise
- ✅ **Noise schedules:** Control the rate of information destruction/reconstruction
- ✅ **Stochasticity:** Random noise additions create variety in generations
- ✅ **Scaling:** More data + bigger models = dramatically better quality
- ✅ **Text conditioning:** Guides generation toward specific concepts
- ✅ **Real-world impact:** Powers DALL-E, Midjourney, Stable Diffusion, and more

### **Connection to Lab 7 (CNNs):**

```
Lab 7 (CNNs):      Image → Features → Classification
                   (Analysis: "What is this?")

Lab 8 (Diffusion): Noise → Features → Image
                   (Synthesis: "Create this!")
```

**Both use convolutional architectures—one for understanding, one for creating!**

---

## 🌟 **Final Thought**

**From noise to images, you've mastered the technology behind DALL-E!** 🎨

You started with pure random noise and learned to gradually sculpt it into recognizable images—the same process used by the most advanced AI image generators in the world.

The only difference between your toy model and DALL-E is **scale**. You have the **knowledge**. With more computational resources, more data, and more training time, you could build the next generation of creative AI tools!

**What's next?**
- Explore Stable Diffusion and try generating your own images
- Learn about other generative models (GANs, VAEs)
- Study advanced diffusion techniques (latent diffusion, classifier-free guidance)
- Build your own creative AI applications

**Thank you for completing Lab 8!** 🚀

---